In [21]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
from datasets import concatenate_datasets
from pysentimiento.irony import load_datasets, id2label

df = pd.read_table("../../data/irony/idpt/Training/training_tweets.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
df = df.rename(columns={"prediction": "label"})

df

,text,label
0,"Para TIM, mudanças regulatórias devem acontece...",0
1,"Quando cheguei a Montemor, gozavam comigo por ...",1
2,"“: Trepidante liderança"" Foi , Marisa BBB15” E...",1
3,Sim sim... nego não sabe diferenciar... você...,1
4,Obg Rosa !! Mas Mariza n faz fofoca kkkk tem ...,1
...,...,...
15207,"então , se TODO mundo é contra a corrupção, co...",1
15208,Economia - A NOVA ECONOMIA. [fonte: Estadão],0
15209,Maioria dos membros da Fed é favorável a uma n...,0
15210,Míriam Leitão: alta do PIB mostra que recessão...,0


In [23]:
# Train dev test split 
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)
train, dev = train_test_split(train, test_size=0.2, random_state=42)

train.shape, dev.shape, test.shape

((9735, 2), (2434, 2), (3043, 2))

In [24]:
from datasets import Dataset, Value, ClassLabel, Features, DatasetDict

features = Features({
    'text': Value('string'),
    'label': ClassLabel(num_classes=2, names=["non ironic", "ironic"]),
})


train = Dataset.from_pandas(train, features=features, preserve_index=False)
dev = Dataset.from_pandas(dev, features=features, preserve_index=False)
test = Dataset.from_pandas(test, features=features, preserve_index=False)

ds = DatasetDict(
    train=train,
    dev=dev,
    test=test
)

ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9735
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 2434
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3043
    })
})

In [25]:
ds.push_to_hub("pysentimiento/pt_irony", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
# Get unique texts in df 

df.text.nunique()

15212